In [8]:
import pandas as pd
from database.market import Market
from database.sec import SEC
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as plt
from processor.processor import Processor as p
import re

In [9]:
market = Market()
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [13]:
sec = SEC()
year = 2022
quarter = 2

In [14]:
whut=['Assets',
 'LiabilitiesAndStockholdersEquity',
 'filed',
 'year',
 'quarter',
 'EarningsPerShareBasic',
 'EarningsPerShareDiluted',
 'StockholdersEquity',
 'NetIncomeLoss',
 'AssetsCurrent',
 'LiabilitiesCurrent']

In [ ]:
market.connect()
sec.connect()
for ticker in tqdm(sp5["Symbol"]):
    try:
        cik = sp5[sp5["Symbol"]==ticker]["CIK"].item()
        filings = sec.retrieve_filing_data(int(cik))
        filings.fillna(0,inplace=True)
        filing_included_columns = [x for x in filings.columns if x in whut]
        aggregated_filings = filings[whut]
        aggregated_filings["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in filings["filed"]]    
        stock_price = market.retrieve_ticker_prices("prices",ticker)
        stock_price = p.column_date_processing(stock_price)
        stock_price.dtypes
        stock_price["year"] = [x.year for x in stock_price["date"]]
        stock_price["quarter"] = [x.quarter for x in stock_price["date"]]
        stock_price = stock_price.groupby(["year","quarter"]).mean().reset_index()
        aggregated_filings["year"] = [x.year for x in aggregated_filings["date"]]
        aggregated_filings["quarter"] = [x.quarter for x in aggregated_filings["date"]]
        ds = aggregated_filings.merge(stock_price[["year","quarter","adjclose"]],on=["year","quarter"])
        ds.drop("filed",inplace=True,axis=1)
        ds = ds[(ds["year"]==year) & (ds["quarter"]==quarter)]
        ds["ticker"] = ticker
        market.store("high_level_filings",ds)
    except Exception as e:
        print(ticker,str(e))
market.disconnect()
sec.disconnect()

  0%|                                                  | 0/504 [00:00<?, ?it/s]C:\Users\eric\AppData\Local\Temp/ipykernel_6480/2644742056.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aggregated_filings["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in filings["filed"]]
C:\Users\eric\AppData\Local\Temp/ipykernel_6480/2644742056.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aggregated_filings["year"] = [x.year for x in aggregated_filings["date"]]
C:\Users\eric\AppData\Local\Temp/ipykernel_

In [ ]:
ds.corr()

In [ ]:
for col in whut:
    try:
        x = ds["date"]
        y1 = ds[col]/100
        y2 = ds["adjclose"]
        fig, ax1 = plt.subplots()
        ax2 = ax1.twinx()
        ax1.plot(x, y1, 'g-')
        ax2.plot(x, y2, 'b-')
        plt.title(col)
        plt.show()
    except:
        continue